In [ ]:
import os
import sys
import subprocess
import jinja2
import requests
from metadata_extractor import CORDEXMetadataExtractor, obs4MIPSMetadataExtractor
from tqdm import tqdm_notebook as tqdm
from glob import glob
from IPython.display import Markdown, Image, FileLink

domain should be one of these three: 'AFR-44', 'EUR-11', 'NAM-44'
* AFR-44: CORDEX Africa RCMs at 44 km resolution
* EUR-11: CORDEX Europe RCMs at 11 km resolution
* NAM-44: CORDEX North America RCMs at 44 km resolution

In [ ]:
domain = 'NAM-44'

In [ ]:
# The output directory
cwd = os.getcwd()
workdir = cwd +'/evaluation_result'

In [ ]:
# Location of obs4Mips files
obs_dir = '/mnt/efs/obs4mips'

In [ ]:
# Location of CORDEX files
models_dir = '/mnt/efs/'+domain+'/*'.format(domain=domain)

In [ ]:
# Extract metadata from model and obs files, pairing up files with the same
# variables for separate evaluations
obs_extractor = obs4MIPSMetadataExtractor(obs_dir)
models_extractor = CORDEXMetadataExtractor(models_dir)
groups = obs_extractor.group(models_extractor, 'variable')

# Configuration file template, to be rendered repeatedly for each evaluation
# run
env =  jinja2.Environment(loader=jinja2.FileSystemLoader('./templates'),
                          trim_blocks=True, lstrip_blocks=True)
t = env.get_template('CORDEX.yaml.template')

# Each group represents a single evaluation. Repeat the evaluation for
# three seasons: Summer, Winter, and Annual.
seasons = ['annual', 'winter', 'summer']
errored = []
for group in tqdm(groups, desc='variable loop'):
    obs_info, models_info = group
    instrument = obs_info['instrument']
    variable = obs_info['variable']
    for season in tqdm(seasons, desc='season loop'):
        configfile_basename = '_'.join([domain, instrument, variable, season]) + '.yaml'
        configfile_path = os.path.join(workdir, domain, instrument,
                                       variable, season)
        if not os.path.exists(configfile_path):
            os.makedirs(configfile_path)
        configfile_path = os.path.join(configfile_path, configfile_basename)
        with open(configfile_path, 'w') as configfile:
            configfile.write(t.render(obs_info=obs_info, models_info=models_info,
                                      season=season, output_dir=workdir))

        # TODO: Do this in parallel. Will change this once this approach
        # is well tested.
        code = subprocess.call([sys.executable, '../run_RCMES.py', configfile_path])
        if code:
            errored.append(configfile_path)
print("All runs done. The following ended with an error: {}".format(errored))

Check the evaluation result or download the processed obs4mips and model output.

In [ ]:
display(Markdown('Evaluation results'))
ip_address = (requests.get('http://169.254.169.254/latest/meta-data/public-ipv4').content).decode('utf-8')
for obs_info in obs_extractor.data:
    inst_name = obs_info['instrument']
    var_name = obs_info['variable']
    display(Markdown('Instrument: '+inst_name+'& Variable: '+var_name))
    for season in seasons:
        savedir = os.path.join('evaluation_result', domain, inst_name, var_name, season)
        png_files = glob(os.path.join(savedir, '*.png'))
        for png_file in png_files:    
            display(Image(png_file))
        nc_file = glob(os.path.join(savedir, '*.nc'))[0]
        display(FileLink(nc_file))
os.chdir(cwd)              
